In [ ]:
import pandas as pd

In [ ]:
df_2024 = pd.read_csv("../data/raw/2023-10-14_2024-10-11_DataExport.csv", delimiter=";")
df_2023 = pd.read_csv("../data/raw/2022-10-14_2023-10-14_DataExport.csv", delimiter=";")
df_2022 = pd.read_csv("../data/raw/2021-10-14_2022-10-14_DataExport.csv", delimiter=";")

In [ ]:
print(df_2024.shape)
print(df_2023.shape)
print(df_2022.shape)

In [ ]:
df = pd.concat([df_2024, df_2023, df_2022], axis=0, ignore_index=True)
print(df.shape)
df.head()


In [ ]:
df = df.drop_duplicates()
df.shape

In [ ]:
df.to_csv("../data/processed/data.csv", sep=";", index=False)

# Remove names and create a single column for identifying the tank

In [ ]:
df = pd.read_csv("../data/processed/data.csv", delimiter=";")

In [ ]:
columns_to_drop = ["Anlage", "Messstellen-TAG", "Produkt", "Kunde", "Straße", "Ort", "Region", "PLZ", "Gerätename", "Geräte-TAG", "Gerätenetz S/N"]
for c in df.columns: 
    unique_values = df[c].unique()
    if len(unique_values) == 1:
        columns_to_drop.append(c)
df = df.drop(columns_to_drop, axis=1)
print(df.columns)

In [ ]:
df['Tank-ID'], uniques = pd.factorize(df['Tankname'])
mapping_dict = dict(enumerate(uniques))
print(mapping_dict)
df.drop("Tankname", axis=1, inplace=True)
df.head()

In [ ]:
mapping_unit_to_column = {
    "Füllstand": "l", 
    "Linear Prozentwert": "%", 
    "Sensorwert": "m", 
    "Leerstand": "l", 
    "Maximale Füllgrenze": "l", 
    "Temperatur": "°C", 
    "Sensorlage": "°"
}

### Remove the units of the table

In [ ]:
for column in ["Füllstand", "Leerstand", "Maximale Füllgrenze"]:
    df[column] = df[column].str.replace(' l', '').str.replace(' Liter', '').str.replace(',', '').astype(float)
for column in ["Linear Prozentwert"]:
    df[column] = df[column].str.replace(' %', '').str.replace(',', '').astype(float)
for column in ["Sensorwert"]:
    df[column] = df[column].str.replace(' m', '').str.replace(',', '').astype(float)
for column in ["Temperatur"]:
    df[column] = df[column].str.replace(' °C', '').str.replace(',', '').astype(float)
for column in ["Sensorlage"]:
    df[column] = df[column].str.replace(' °', '').str.replace(',', '').astype(float)

# Not used because of negative values and the - would be removed
#for column in ["Linear Prozentwert", "Sensorwert", "Temperatur", "Sensorlage"]: 
#    df[column] = df[column].str.replace(r'\D', '', regex=True).astype(float)
df.head()

In [ ]:
df.to_csv("../data/processed/data_cleaned.csv", sep=";", index=False)

# One datapoint per day

Group per tank and day and calculate average of that day (for non-numeric features select min)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/processed/data_cleaned.csv", delimiter=";")
df.head()

In [ ]:
df["Zeitstempel"] = pd.to_datetime(df['Zeitstempel']).dt.date
df.sort_values("Zeitstempel").head()

In [ ]:
aggregation_functions = {col: 'min' for col in df.select_dtypes(include='number').columns}
aggregation_functions.update({col: 'min' for col in df.select_dtypes(include='object').columns})
del aggregation_functions["Tank-ID"]
del aggregation_functions["Zeitstempel"]
print(aggregation_functions)

# Group by 'tank-id' and 'date', apply the aggregation functions
df = df.groupby(['Tank-ID', 'Zeitstempel']).agg(aggregation_functions).reset_index()
df.head()

In [ ]:
df.to_csv("../data/processed/data_one_day.csv", sep=";", index=False)

Check if there are gaps in the data

In [ ]:
tank_ids = df["Tank-ID"].unique()
for id in tank_ids:
    full_range = pd.date_range(start=df[df['Tank-ID'] == id]['Zeitstempel'].min(), end=df['Zeitstempel'].max())

    # Find the missing dates by checking which dates in the full range are not in the DataFrame
    missing_dates = full_range.difference(df['Zeitstempel'])

    print(f"Tank-ID {id}: Number of missing dates: {len(missing_dates)}")

# Calculate 'Verbrauch' per day

In [ ]:
df = df.sort_values(by=['Tank-ID', 'Zeitstempel'])

# Group by 'Tank-ID' and calculate the difference in 'Füllstand' for each day
df['Verbrauch'] = df.groupby('Tank-ID')['Füllstand'].diff().shift(-1)

# Remove outliers

At first manual, to get clean data. 
If there is time afterwards make it automatically to generalize it. 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 9))
for tank_id, ax in zip(df["Tank-ID"].unique().tolist(), axes.flatten()):
    df_temp = df[df["Tank-ID"] == tank_id]
    ax.set_title(f"Tank-ID {tank_id}")
    sns.lineplot(data=df_temp, y="Füllstand", x="Zeitstempel", ax=ax)
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 9))
for tank_id, ax in zip(df["Tank-ID"].unique().tolist(), axes.flatten()):
    df_temp = df[df["Tank-ID"] == tank_id]
    ax.set_title(f"Tank-ID {tank_id}")
    sns.lineplot(data=df_temp, y="Verbrauch", x="Zeitstempel", ax=ax)
fig.tight_layout()

To get the outliers there I try the following approaches:
- Z-score
- IQR
- Rolling Window

Other more advanced methods could be ML models

In [ ]:
# Simple threshold
df["Threshold Outlier"] = np.nan
df["Threshold Outlier"] = df["Verbrauch"].abs() > 20

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(12, 15))
scatter = True
duplicate_tank_ids = sorted([id for id in tank_ids for _ in range(2)])
for tank_id, ax in zip(duplicate_tank_ids, axes.flatten()):
    df_temp = df[df["Tank-ID"] == tank_id]
    ax.set_title(f"Tank-ID {tank_id}")
    if scatter:
        sns.scatterplot(data=df_temp, y="Verbrauch", x="Zeitstempel", hue="Threshold Outlier", ax=ax)
        scatter = False
    else:
        sns.lineplot(data=df_temp[df_temp["Threshold Outlier"] == False], y="Verbrauch", x="Zeitstempel", ax=ax)
        scatter = True
fig.tight_layout()

In [ ]:
# Z-Score Outlier
df["Z-Score"] = np.nan
for id in tank_ids:
    df.loc[df["Tank-ID"] == id, "Z-Score"] = (df[df["Tank-ID"] == id]['Verbrauch'] - df[df["Tank-ID"] == id]['Verbrauch'].mean()) / df[df["Tank-ID"] == id]['Verbrauch'].std()

print(df["Z-Score"].isna().sum())
# df[df["Z-Score"].isna()].head(10)
# df.describe()
df["Z-Score"].fillna(0)
df["Z-Score Outlier"] = None
df["Z-Score Outlier"] = df["Z-Score"].abs() > 2

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(12, 15))
scatter = True
duplicate_tank_ids = sorted([id for id in tank_ids for _ in range(2)])
for tank_id, ax in zip(duplicate_tank_ids, axes.flatten()):
    df_temp = df[df["Tank-ID"] == tank_id]
    ax.set_title(f"Tank-ID {tank_id}")
    if scatter:
        sns.scatterplot(data=df_temp, y="Verbrauch", x="Zeitstempel", hue="Threshold Outlier", ax=ax)
        scatter = False
    else:
        sns.lineplot(data=df_temp[df_temp["Z-Score Outlier"] == False], y="Verbrauch", x="Zeitstempel", ax=ax)
        scatter = True
fig.tight_layout()

In [ ]:
# IQR Outliers
df["IQR Outlier"] = None
for id in tank_ids:
    Q1 = df[df["Tank-ID"] == id]['Verbrauch'].quantile(0.25)
    Q3 = df[df["Tank-ID"] == id]['Verbrauch'].quantile(0.75)
    IQR = Q3 - Q1

    # Define outlier bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    lb = df[(df["Tank-ID"] == id)]["Verbrauch"] < lower_bound 
    ub = df[(df["Tank-ID"] == id)]["Verbrauch"] > upper_bound

    df.loc[df["Tank-ID"] == id, "IQR Outlier"] = lb | ub
    #df.loc[df["Tank-ID"] == id, "IQR Outlier"] = df[df["Tank-ID"] == id][(df[df["Tank-ID"] == id]['Verbrauch'] < lower_bound) | (df[df["Tank-ID"] == id]['Verbrauch'] > upper_bound)]

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(12, 15))
scatter = True
duplicate_tank_ids = sorted([id for id in tank_ids for _ in range(2)])
for tank_id, ax in zip(duplicate_tank_ids, axes.flatten()):
    df_temp = df[df["Tank-ID"] == tank_id]
    ax.set_title(f"Tank-ID {tank_id}")
    if scatter:
        sns.scatterplot(data=df_temp, y="Verbrauch", x="Zeitstempel", hue="Threshold Outlier", ax=ax)
        scatter = False
    else:
        sns.lineplot(data=df_temp[df_temp["IQR Outlier"] == False], y="Verbrauch", x="Zeitstempel", ax=ax)
        scatter = True
fig.tight_layout()

In [ ]:
# Rolling Window Outliers
df["RW Outlier"] = None
for id in tank_ids:
    # Use a rolling window to detect peaks
    df['RW Outlier'] = (df['Verbrauch'] > df['Verbrauch'].shift(2)) & (df['Verbrauch'] > df['Verbrauch'].shift(1)) & (df['Verbrauch'] > df['Verbrauch'].shift(-1)) & (df['Verbrauch'] > df['Verbrauch'].shift(-2))

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=2, figsize=(12, 15))
scatter = True
duplicate_tank_ids = sorted([id for id in tank_ids for _ in range(2)])
for tank_id, ax in zip(duplicate_tank_ids, axes.flatten()):
    df_temp = df[df["Tank-ID"] == tank_id]
    ax.set_title(f"Tank-ID {tank_id}")
    if scatter:
        sns.scatterplot(data=df_temp, y="Verbrauch", x="Zeitstempel", hue="Threshold Outlier", ax=ax)
        scatter = False
    else:
        sns.lineplot(data=df_temp[df_temp["RW Outlier"] == False], y="Verbrauch", x="Zeitstempel", ax=ax)
        scatter = True
fig.tight_layout()

With the IQR method the outliers are detected. 
We interpolate the detected outliers with the mean of the day before and after. 

## Replace the outliers with interpolation

In [ ]:
dfs_per_id = []
for id in tank_ids:
    df_tmp = df[df["Tank-ID"] == id]
    df_tmp = df_tmp.sort_values(by='Zeitstempel').reset_index(drop=True)
    for i in range(len(df_tmp)):
        if df_tmp.loc[i, 'IQR Outlier']:
            # Find last valid value before the outlier
            #last_valid = df_tmp.loc[:i - 1, 'Verbrauch'][~df['IQR Outlier']].tail(1)
            last_valid = df_tmp.loc[:(i - 1)]
            last_valid = last_valid[last_valid["IQR Outlier"] == False]
            last_valid = last_valid.tail(1)["Verbrauch"]

            # Find first valid value after the outlier
            #first_valid = df_tmp.loc[i + 1:, 'Verbrauch'][~df['IQR Outlier']].head(1)
            first_valid = df_tmp.loc[(i + 1):]
            first_valid = first_valid[first_valid["IQR Outlier"] == False]
            first_valid = first_valid.head(1)["Verbrauch"]

            # Calculate the mean of the last valid and first valid values
            if not last_valid.empty and not first_valid.empty:
                new_value = (last_valid.values[0] + first_valid.values[0]) / 2
                df_tmp.loc[i, 'Verbrauch'] = new_value
            if not last_valid.empty and first_valid.empty:
                new_value = (last_valid.values[0])
                df_tmp.loc[i, 'Verbrauch'] = new_value
            if not first_valid.empty and last_valid.empty:
                new_value = (first_valid.values[0])
                df_tmp.loc[i, 'Verbrauch'] = new_value
    
    dfs_per_id.append(df_tmp)

df_new = pd.concat(dfs_per_id, axis=0, ignore_index=True)
df = df_new[~df_new["Verbrauch"].isna()]

In [ ]:
df.head()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 9))
for tank_id, ax in zip(df_new["Tank-ID"].unique().tolist(), axes.flatten()):
    df_temp = df_new[df_new["Tank-ID"] == tank_id]
    ax.set_title(f"Tank-ID {tank_id}")
    sns.lineplot(data=df_temp, y="Verbrauch", x="Zeitstempel", ax=ax)
fig.tight_layout()

In [ ]:
df_new.to_parquet("../data/processed/data.parquet")
df_new.to_pickle("../data/processed/data.pickle")


# Get baseline from noisy values

In [ ]:
from scipy.ndimage import uniform_filter1d
from pybaselines import Baseline

In [ ]:
df_new = df_new[~df_new["Verbrauch"].isna()]
print(df_new["Verbrauch"].isna().sum())

In [ ]:
df_new["Verbrauch smoothed"] = np.nan
dfs_per_id = []
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 9))
for tank_id, ax in zip(df_new["Tank-ID"].unique().tolist(), axes.flatten()):
    df_temp = df_new[df_new["Tank-ID"] == tank_id]
    df_temp.loc[:, "Verbrauch smoothed"] = uniform_filter1d(df_temp["Verbrauch"], 11)
    dfs_per_id.append(df_temp)
    #baseline_fitter = Baseline(x_data=df_temp["Verbrauch"])
    #df_temp["smoothed_modpoly"] = baseline_fitter.imodpoly(df_temp["Verbrauch"], poly_order=3, num_std=0.7)[0]

    ax.set_title(f"Tank-ID {tank_id}")
    sns.lineplot(y=df_temp["Verbrauch"], x=df_temp["Zeitstempel"], color='blue', ax=ax)
    sns.lineplot(y=df_temp["Verbrauch smoothed"], x=df_temp["Zeitstempel"], color='orange', ax=ax)
    #sns.lineplot(y=df_temp["smoothed_modpoly"], x=df_temp["Zeitstempel"], color='green', ax=ax)
fig.tight_layout()

df = pd.concat(dfs_per_id, axis=0, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.drop(["Threshold Outlier", "Z-Score", "Z-Score Outlier", "IQR Outlier", "RW Outlier"], axis=1, inplace=True)
df.head()

In [ ]:
df.to_pickle("../data/processed/data_one_day_clean.pickle")

# Verify data